# Policy gradients


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.7607606144250734 -2.992332324761291
actions: 1 0
rewards: 1.0 1.0


In [8]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [9]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [11]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [12]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [13]:
# Data of the model
def model_input(state_size):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    reward = tf.placeholder(tf.float32, [], name='reward')
    return states, actions, targetQs, reward

In [14]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [15]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [16]:
def model_loss(action_size, hidden_size, states, actions, targetQs, reward):
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    
    Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    d_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits,
                                                                    labels=reward*tf.ones_like(Qs_logits)))
    # tf.nn.sigmoid(1+0.99*(Qs_logits)) --> tf.ones_like(Qs_logits) if Qs_logits >> 1
    #Qs_label = targetQs[1:]
    # g_lossQ = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits,
    #                                                                  labels=tf.ones_like(Qs_logits)))
    g_lossQ = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits[:-1],
                                                                     labels=tf.reshape(
                                                                         tf.nn.sigmoid(targetQs[1:]), 
                                                                         shape=[-1, 1])))
    #g_lossP = tf.reduce_mean(neg_log_prob_actions * tf.nn.sigmoid(targetQs))
    #g_lossP = tf.reduce_mean(neg_log_prob_actions * targetQs)
    g_lossP = tf.reduce_mean(neg_log_prob_actions[:-1] * targetQs[1:])
    #g_loss = g_lossQ + g_lossP
    g_loss = g_lossP

    return actions_logits, Qs_logits, g_loss, d_loss, g_lossQ, g_lossP

In [17]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [18]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.reward = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss, self.g_lossQ, self.g_lossP = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, # model input
            targetQs=self.targetQs, reward=self.reward) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [19]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [20]:
# Training parameters
# Network parameters
state_size = 4               # number of units for the input state/observation -- simulation
action_size = 2              # number of units for the output actions -- simulation
hidden_size = 64             # number of units in each Q-network hidden layer -- simulation
learning_rate = 0.001          # learning rate for adam

In [21]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [22]:
# import gym

# ## Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
# env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# # env = gym.make('MountainCar-v0')
# # env = gym.make('Pendulum-v0')
# # env = gym.make('Blackjack-v0')
# # env = gym.make('FrozenLake-v0')
# # env = gym.make('AirRaid-ram-v0')
# # env = gym.make('AirRaid-v0')
# # env = gym.make('BipedalWalker-v2')
# # env = gym.make('Copy-v0')
# # env = gym.make('CarRacing-v0')
# # env = gym.make('Ant-v2') #mujoco
# # env = gym.make('FetchPickAndPlace-v1') # mujoco required!

In [ ]:
from collections import deque
episodes_total_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(11111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state

        # Training steps/batches
        while True:
            action_logits, Q_logits = sess.run(fetches=[model.actions_logits, model.Qs_logits], 
                                               feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            batch.append([state, action, Q_logits])
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done is True: # episode ended success/failure
                episodes_total_reward.append(total_reward) # stopping criteria
                rate = total_reward/ 500 # success is 500 points
                break

        # Training using batches
        #batch = memory.buffer
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        targetQs = np.array([each[2] for each in batch])
        g_lossQ, g_lossP, g_loss, d_loss, _, _ = sess.run([model.g_lossQ, model.g_lossP, 
                                                           model.g_loss, model.d_loss, 
                                                           model.g_opt, model.d_opt],
                                                          feed_dict = {model.states: states, 
                                                                       model.actions: actions,
                                                                       model.reward: rate,
                                                                       model.targetQs: targetQs.reshape([-1])})
        # Average 100 episode total reward
        # Print out
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episodes_total_reward)),
              'glossQ:{:.4f}'.format(g_lossQ),
              'glossP:{:.4f}'.format(g_lossP),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss))
        # Ploting out
        rewards_list.append([ep, np.mean(episodes_total_reward)])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        # Break episode/epoch loop
        if np.mean(episodes_total_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:52.0000 glossQ:0.6931 glossP:0.0158 gloss:0.0158 dloss:0.7023
Episode:1 meanR:39.0000 glossQ:0.6932 glossP:0.0007 gloss:0.0007 dloss:0.6934
Episode:2 meanR:29.3333 glossQ:0.6929 glossP:-0.0272 gloss:-0.0272 dloss:0.6743
Episode:3 meanR:25.0000 glossQ:0.6921 glossP:-0.0624 gloss:-0.0624 dloss:0.6523
Episode:4 meanR:22.8000 glossQ:0.6910 glossP:-0.0890 gloss:-0.0890 dloss:0.6363
Episode:5 meanR:21.6667 glossQ:0.6901 glossP:-0.1072 gloss:-0.1072 dloss:0.6254
Episode:6 meanR:26.7143 glossQ:0.6914 glossP:-0.0834 gloss:-0.0834 dloss:0.6485
Episode:7 meanR:28.3750 glossQ:0.6893 glossP:-0.1216 gloss:-0.1216 dloss:0.6240
Episode:8 meanR:30.3333 glossQ:0.6847 glossP:-0.1384 gloss:-0.1384 dloss:0.6207
Episode:9 meanR:28.3000 glossQ:0.6764 glossP:0.2118 gloss:0.2118 dloss:0.8382
Episode:10 meanR:26.7273 glossQ:0.6752 glossP:0.2063 gloss:0.2063 dloss:0.8420
Episode:11 meanR:25.2500 glossQ:0.6841 glossP:0.1380 gloss:0.1380 dloss:0.7879
Episode:12 meanR:24.0769 glossQ:0.6853 glossP:0.

Episode:103 meanR:49.2200 glossQ:0.4065 glossP:-1.2320 gloss:-1.2320 dloss:0.2641
Episode:104 meanR:49.4500 glossQ:0.3996 glossP:-1.2532 gloss:-1.2532 dloss:0.2838
Episode:105 meanR:49.5000 glossQ:0.3984 glossP:-1.2602 gloss:-1.2602 dloss:0.2244
Episode:106 meanR:49.1000 glossQ:0.3897 glossP:-1.2862 gloss:-1.2862 dloss:0.2060
Episode:107 meanR:48.8900 glossQ:0.3697 glossP:-1.3545 gloss:-1.3545 dloss:0.2059
Episode:108 meanR:48.6000 glossQ:0.3587 glossP:-1.3902 gloss:-1.3902 dloss:0.1937
Episode:109 meanR:48.6400 glossQ:0.3425 glossP:-1.4461 gloss:-1.4461 dloss:0.1763
Episode:110 meanR:48.6900 glossQ:0.3174 glossP:-1.5353 gloss:-1.5353 dloss:0.1764
Episode:111 meanR:48.7100 glossQ:0.3008 glossP:-1.5992 gloss:-1.5992 dloss:0.1508
Episode:112 meanR:48.7100 glossQ:0.2917 glossP:-1.6345 gloss:-1.6345 dloss:0.1429
Episode:113 meanR:48.7300 glossQ:0.2670 glossP:-1.7374 gloss:-1.7374 dloss:0.1415
Episode:114 meanR:48.7800 glossQ:0.2625 glossP:-1.7582 gloss:-1.7582 dloss:0.1528
Episode:115 mean

Episode:208 meanR:29.4500 glossQ:0.2064 glossP:-1.9927 gloss:-1.9927 dloss:0.1475
Episode:209 meanR:29.5000 glossQ:0.2209 glossP:-1.9268 gloss:-1.9268 dloss:0.1663
Episode:210 meanR:29.4700 glossQ:0.1964 glossP:-2.0437 gloss:-2.0437 dloss:0.1289
Episode:211 meanR:29.4800 glossQ:0.1919 glossP:-2.0748 gloss:-2.0748 dloss:0.1223
Episode:212 meanR:29.5200 glossQ:0.2013 glossP:-2.0313 gloss:-2.0313 dloss:0.1356
Episode:213 meanR:29.5500 glossQ:0.1884 glossP:-2.0993 gloss:-2.0993 dloss:0.1336
Episode:214 meanR:29.5900 glossQ:0.1901 glossP:-2.0925 gloss:-2.0925 dloss:0.1575
Episode:215 meanR:29.6100 glossQ:0.1815 glossP:-2.1404 gloss:-2.1404 dloss:0.1208
Episode:216 meanR:29.6700 glossQ:0.2097 glossP:-2.0018 gloss:-2.0018 dloss:0.1599
Episode:217 meanR:29.7600 glossQ:0.2520 glossP:-1.7968 gloss:-1.7968 dloss:0.3175
Episode:218 meanR:29.6600 glossQ:0.2428 glossP:-1.8334 gloss:-1.8334 dloss:0.2385
Episode:219 meanR:29.9400 glossQ:0.2272 glossP:-1.8990 gloss:-1.8990 dloss:0.3990
Episode:220 mean

Episode:310 meanR:41.4500 glossQ:0.2551 glossP:-1.7941 gloss:-1.7941 dloss:0.1567
Episode:311 meanR:41.6200 glossQ:0.2715 glossP:-1.7259 gloss:-1.7259 dloss:0.2246
Episode:312 meanR:41.7400 glossQ:0.2646 glossP:-1.7536 gloss:-1.7536 dloss:0.2093
Episode:313 meanR:41.8800 glossQ:0.2531 glossP:-1.8011 gloss:-1.8011 dloss:0.2188
Episode:314 meanR:42.4600 glossQ:0.2848 glossP:-1.6706 gloss:-1.6706 dloss:0.4542
Episode:315 meanR:42.6900 glossQ:0.2509 glossP:-1.8160 gloss:-1.8160 dloss:0.2562
Episode:316 meanR:43.0000 glossQ:0.2397 glossP:-1.8655 gloss:-1.8655 dloss:0.3322
Episode:317 meanR:43.0600 glossQ:0.3105 glossP:-1.5752 gloss:-1.5752 dloss:0.3406
Episode:318 meanR:43.3200 glossQ:0.3034 glossP:-1.5992 gloss:-1.5992 dloss:0.3639
Episode:319 meanR:43.0100 glossQ:0.2920 glossP:-1.6427 gloss:-1.6427 dloss:0.2324
Episode:320 meanR:43.1700 glossQ:0.2845 glossP:-1.6682 gloss:-1.6682 dloss:0.3327
Episode:321 meanR:43.3300 glossQ:0.2839 glossP:-1.6718 gloss:-1.6718 dloss:0.3135
Episode:322 mean

Episode:410 meanR:70.4500 glossQ:0.4389 glossP:-1.1553 gloss:-1.1553 dloss:0.2509
Episode:411 meanR:70.3800 glossQ:0.4249 glossP:-1.1983 gloss:-1.1983 dloss:0.2407
Episode:412 meanR:70.3200 glossQ:0.4207 glossP:-1.2113 gloss:-1.2113 dloss:0.2317
Episode:413 meanR:70.2800 glossQ:0.4020 glossP:-1.2686 gloss:-1.2686 dloss:0.2376
Episode:414 meanR:69.7100 glossQ:0.3963 glossP:-1.2898 gloss:-1.2898 dloss:0.2175
Episode:415 meanR:69.5300 glossQ:0.3807 glossP:-1.3396 gloss:-1.3396 dloss:0.2034
Episode:416 meanR:69.3000 glossQ:0.3649 glossP:-1.3909 gloss:-1.3909 dloss:0.2325
Episode:417 meanR:68.9600 glossQ:0.3439 glossP:-1.4612 gloss:-1.4612 dloss:0.1978
Episode:418 meanR:68.5100 glossQ:0.3306 glossP:-1.5105 gloss:-1.5105 dloss:0.1706
Episode:419 meanR:68.3800 glossQ:0.3176 glossP:-1.5576 gloss:-1.5576 dloss:0.1817
Episode:420 meanR:68.0100 glossQ:0.2870 glossP:-1.6698 gloss:-1.6698 dloss:0.1589
Episode:421 meanR:67.6700 glossQ:0.2769 glossP:-1.7129 gloss:-1.7129 dloss:0.1523
Episode:422 mean

...

Episode:7334 meanR:140.8000 glossQ:0.5995 glossP:-0.6272 gloss:-0.6272 dloss:0.5594
Episode:7335 meanR:140.3900 glossQ:0.5934 glossP:-0.6484 gloss:-0.6484 dloss:0.5976
Episode:7336 meanR:139.7000 glossQ:0.5882 glossP:-0.6674 gloss:-0.6674 dloss:0.5678
Episode:7337 meanR:139.5800 glossQ:0.5646 glossP:-0.7489 gloss:-0.7489 dloss:0.5377
Episode:7338 meanR:139.2200 glossQ:0.5373 glossP:-0.8351 gloss:-0.8351 dloss:0.4996
Episode:7339 meanR:138.7500 glossQ:0.4921 glossP:-0.9889 gloss:-0.9889 dloss:0.4535
Episode:7340 meanR:137.9800 glossQ:0.4725 glossP:-1.0615 gloss:-1.0615 dloss:0.3882
Episode:7341 meanR:137.1200 glossQ:0.2822 glossP:-1.8248 gloss:-1.8248 dloss:0.1644
Episode:7342 meanR:137.6900 glossQ:0.5881 glossP:-0.6680 gloss:-0.6680 dloss:0.5986
Episode:7343 meanR:141.1800 glossQ:0.6282 glossP:-0.5306 gloss:-0.5306 dloss:1.0009
Episode:7344 meanR:141.3700 glossQ:0.5191 glossP:-1.0397 gloss:-1.0397 dloss:0.4252
Episode:7345 meanR:141.4400 glossQ:0.4279 glossP:-1.5220 gloss:-1.5220 dloss

Episode:7432 meanR:162.4600 glossQ:0.6921 glossP:0.1776 gloss:0.1776 dloss:0.5782
Episode:7433 meanR:166.1200 glossQ:0.6785 glossP:0.2522 gloss:0.2522 dloss:0.5427
Episode:7434 meanR:169.9400 glossQ:0.6895 glossP:0.2144 gloss:0.2144 dloss:0.5550
Episode:7435 meanR:172.7900 glossQ:0.6863 glossP:0.1713 gloss:0.1713 dloss:0.6192
Episode:7436 meanR:174.8000 glossQ:0.6968 glossP:0.0222 gloss:0.0222 dloss:0.6925
Episode:7437 meanR:175.8500 glossQ:0.6948 glossP:0.0142 gloss:0.0142 dloss:0.6977
Episode:7438 meanR:176.8100 glossQ:0.6931 glossP:-0.0739 gloss:-0.0739 dloss:0.6856
Episode:7439 meanR:177.7700 glossQ:0.6872 glossP:-0.1332 gloss:-0.1332 dloss:0.6749
Episode:7440 meanR:178.6500 glossQ:0.6837 glossP:-0.1549 gloss:-0.1549 dloss:0.6572
Episode:7441 meanR:179.9300 glossQ:0.6789 glossP:-0.1933 gloss:-0.1933 dloss:0.6453
Episode:7442 meanR:179.6400 glossQ:0.6832 glossP:-0.1775 gloss:-0.1775 dloss:0.6342
Episode:7443 meanR:176.9300 glossQ:0.6702 glossP:-0.2731 gloss:-0.2731 dloss:0.6250
Epis

Episode:7530 meanR:117.8600 glossQ:0.5549 glossP:-0.7926 gloss:-0.7926 dloss:0.5813
Episode:7531 meanR:116.3900 glossQ:0.6136 glossP:-0.5818 gloss:-0.5818 dloss:0.6176
Episode:7532 meanR:113.1000 glossQ:0.6426 glossP:-0.4524 gloss:-0.4524 dloss:0.6476
Episode:7533 meanR:110.3100 glossQ:0.6600 glossP:-0.3690 gloss:-0.3690 dloss:0.6938
Episode:7534 meanR:109.0400 glossQ:0.6683 glossP:-0.3355 gloss:-0.3355 dloss:0.8509
Episode:7535 meanR:109.8000 glossQ:0.6803 glossP:-0.3288 gloss:-0.3288 dloss:0.9711
Episode:7536 meanR:110.9300 glossQ:0.6390 glossP:-0.4728 gloss:-0.4728 dloss:1.0236
Episode:7537 meanR:112.1400 glossQ:0.6261 glossP:-0.5135 gloss:-0.5135 dloss:0.8991
Episode:7538 meanR:114.2700 glossQ:0.6169 glossP:-0.5627 gloss:-0.5627 dloss:1.0244
Episode:7539 meanR:114.5000 glossQ:0.5929 glossP:-0.6346 gloss:-0.6346 dloss:0.7114
Episode:7540 meanR:114.7800 glossQ:0.5803 glossP:-0.6798 gloss:-0.6798 dloss:0.6700
Episode:7541 meanR:114.6900 glossQ:0.5846 glossP:-0.6719 gloss:-0.6719 dloss

Episode:7629 meanR:120.2000 glossQ:0.6132 glossP:-0.5666 gloss:-0.5666 dloss:0.5451
Episode:7630 meanR:119.8100 glossQ:0.6070 glossP:-0.5808 gloss:-0.5808 dloss:0.5252
Episode:7631 meanR:119.1800 glossQ:0.5991 glossP:-0.6218 gloss:-0.6218 dloss:0.5162
Episode:7632 meanR:118.0700 glossQ:0.5524 glossP:-0.7950 gloss:-0.7950 dloss:0.4231
Episode:7633 meanR:116.7700 glossQ:0.5891 glossP:-0.6559 gloss:-0.6559 dloss:0.4973
Episode:7634 meanR:113.5100 glossQ:0.5475 glossP:-0.8147 gloss:-0.8147 dloss:0.3919
Episode:7635 meanR:109.4100 glossQ:0.5874 glossP:-0.6693 gloss:-0.6693 dloss:0.5075
Episode:7636 meanR:105.5300 glossQ:0.5495 glossP:-0.8058 gloss:-0.8058 dloss:0.3958
Episode:7637 meanR:103.1200 glossQ:0.5879 glossP:-0.6748 gloss:-0.6748 dloss:0.5150
Episode:7638 meanR:100.3000 glossQ:0.6463 glossP:-0.3787 gloss:-0.3787 dloss:0.6299
Episode:7639 meanR:99.4000 glossQ:0.5930 glossP:-0.6580 gloss:-0.6580 dloss:0.5425
Episode:7640 meanR:98.5200 glossQ:0.6170 glossP:-0.5474 gloss:-0.5474 dloss:0

Episode:7731 meanR:109.2400 glossQ:0.3514 glossP:-1.4523 gloss:-1.4523 dloss:0.3266
Episode:7732 meanR:108.8100 glossQ:0.2482 glossP:-1.8398 gloss:-1.8398 dloss:0.1686
Episode:7733 meanR:108.1700 glossQ:0.2316 glossP:-1.9772 gloss:-1.9772 dloss:0.1851
Episode:7734 meanR:107.8100 glossQ:0.2221 glossP:-2.1005 gloss:-2.1005 dloss:0.1569
Episode:7735 meanR:107.5000 glossQ:0.3041 glossP:-1.6288 gloss:-1.6288 dloss:0.3792
Episode:7736 meanR:107.6900 glossQ:0.3500 glossP:-1.4618 gloss:-1.4618 dloss:0.4136
Episode:7737 meanR:106.8600 glossQ:0.2027 glossP:-2.2534 gloss:-2.2534 dloss:0.1546
Episode:7738 meanR:105.8300 glossQ:0.2205 glossP:-2.0887 gloss:-2.0887 dloss:0.1611
Episode:7739 meanR:107.1200 glossQ:0.4912 glossP:-1.0149 gloss:-1.0149 dloss:0.9136
Episode:7740 meanR:107.6200 glossQ:0.4953 glossP:-0.9747 gloss:-0.9747 dloss:0.6093
Episode:7741 meanR:106.3600 glossQ:0.2803 glossP:-1.7929 gloss:-1.7929 dloss:0.1689
Episode:7742 meanR:104.1800 glossQ:0.3653 glossP:-1.4089 gloss:-1.4089 dloss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [28]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        if total_reward == 500:
            break
                
# Closing the env
env.close()

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model2.ckpt
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.